In [1]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd

In [2]:
from config import ACCESS_KEY,SECRET_KEY,TOKEN,APP_METRICA_OAUTH

In [3]:
def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    days_to_subtract = yesterday.day - 1
    start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
    yesterday = now - datetime.timedelta(days=1)

    return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'now_date':now.strftime('%Y-%m-%d'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d')

            }

# Переменные, которые могут быть использованы в разоичных запросах. 
now = get_now_datetime_str()['now']
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']

limit = 100
offset = 1

In [4]:
month = '05'
year = '2024'
start_of_month_data = '2024-05-01'
now_date = '2024-06-01'
yesterday_data = '2024-05-31'

In [5]:
headers={'Authorization':APP_METRICA_OAUTH,'Host':'api.appmetrica.yandex.ru'}
limit = 100
offset = 1


def get_request_metrica(api_html_metrica):
    response = requests.get(
        api_html_metrica,
        headers = headers
    )
    return response

In [6]:
api_html_appmetrica_auditorium = open('api_html_appmetrica_auditorium.txt','r').read().format(start_of_month_data,yesterday_data,offset,limit)
response = get_request_metrica(api_html_appmetrica_auditorium)

In [7]:
# Преобразование файлов из 

api_html_appmetrica_auditorium = open('api_html_appmetrica_auditorium.txt','r').read().format(start_of_month_data,yesterday_data,offset,limit)
response = get_request_metrica(api_html_appmetrica_auditorium)

for i in range(0,len(response.json()['data'])):
    if i == 0:
        data_DAU = pd.DataFrame({
            'Дата': response.json()['data'][i]['dimensions'][0]['name'],
            'Пользователи': [int(response.json()['data'][i]['metrics'][0])],
            'Новые пользователи':  [int(response.json()['data'][i]['metrics'][2])],
            'Доля новых пользователей': [response.json()['data'][i]['metrics'][4]]
            })
    else:
        data_DAU_new_row = pd.DataFrame({
            'Дата': response.json()['data'][i]['dimensions'][0]['name'],
            'Пользователи': [int(response.json()['data'][i]['metrics'][0])],
            'Новые пользователи':  [int(response.json()['data'][i]['metrics'][2])],
            'Доля новых пользователей': [response.json()['data'][i]['metrics'][4]]
            })
        data_DAU = pd.concat([data_DAU,data_DAU_new_row])

data_DAU.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_DAU_MAU/remade/DAU/{year}/{int(month)}.csv', index=False, sep=';')

data_MAU = pd.DataFrame({
    'Пользователи': [int(response.json()['totals'][0])],
    'Новые пользователи':  [int(response.json()['totals'][2])],
    'Доля новых пользователей': [response.json()['totals'][4]]
    })

data_MAU.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_DAU_MAU/remade/MAU/{year}/{int(month)}.csv', index=False, sep=';')

In [11]:
# appmetrica_ID_count

api_html_appmetrica_ID_count = open('api_html_appmetrica_ID_count.txt','r').read().format('2019-01-01',yesterday_data,offset,limit)
response = get_request_metrica(api_html_appmetrica_ID_count)

id_count = pd.DataFrame({
    'Android': int(response.json()['data'][0]['metrics'][0][0]),
    'Ios': int(response.json()['data'][1]['metrics'][0][0])
    }
    ,index=[0])

id_count
id_count.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_ID_count/remade/{int(year)}/{int(month)}.csv', index=False, sep=";")

In [12]:
# appmetrica_installetion_count

api_html_appmetrica_installetion_count = open('api_html_appmetrica_installetion_count.txt','r').read().format('2019-01-01',yesterday_data,limit,offset)
response = get_request_metrica(api_html_appmetrica_installetion_count)

installetion_count = pd.DataFrame({
    'Все установки': int(response.json()['totals'][1]),
    'Сессии': int(response.json()['totals'][4])
    }
    ,index=[0])

installetion_count.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_all_daownloaded/remade/{int(year)}/{int(month)}.csv', index=False,sep=";")

In [13]:
# appmetrica_installetion_count_per_day

api_html_appmetrica_installetion_count_per_day = open('api_html_appmetrica_installetion_count_per_day.txt','r').read().format(start_of_month_data,yesterday_data)
response = get_request_metrica(api_html_appmetrica_installetion_count_per_day)

for i in range(0, len(response.json()['totals'][0])):
    if i == 0:
        count_per_day = pd.DataFrame({
            'Дата': response.json()['time_intervals'][i][0],
            'Установки': int(response.json()['totals'][0][i])
            },index=[0])
    else:
        count_per_day_new_row = pd.DataFrame({
            'Дата': response.json()['time_intervals'][i][0],
            'Установки': int(response.json()['totals'][0][i])
            },index=[0])
        count_per_day = pd.concat([count_per_day,count_per_day_new_row])

count_per_day.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_download_by_day/remade/{int(year)}/{int(month)}.csv', index=False,sep=";")

In [5]:
token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3
BUCKET_NAME = 'dwh-asgard' #имя бакета
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"

In [6]:
def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'



def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for i in rows:
            special_str = ""
            for j in i:
                if isinstance(j, str):
                    special_str = f"{special_str}'{str(j)}',"
                else:
                    special_str = f"{special_str}{str(j)},"
            temp_file.write(special_str[:-1]+'\n') 
        offset +=1000 # увеличивает смещение

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

In [16]:
FOLDER = 'total_activated_account_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()


In [18]:
FOLDER = 'total_active_users_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(now_date,start_of_month_data,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [19]:
FOLDER = 'active_users_in_flats_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(now_date,start_of_month_data,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [20]:
FOLDER = 'total_account_count_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [21]:
FOLDER = 'DAF_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [22]:
FOLDER = 'MAF_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [23]:
FOLDER = 'app_in_flats_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [24]:
FOLDER = 'flats_in_sputnic_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [25]:
FOLDER = 'all_flats_rep_mobile'

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [7]:
FOLDER = 'Units_on_SK platform_(monetization)_rep_mobile'

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [27]:
FOLDER = 'paying_active_flats_rep_mobile'

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [28]:
FOLDER = 'subscriptions_report_comerce_rep_mobile'
offset = 0
query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [29]:
FOLDER = 'mobile_report_rep_mobile'

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data,month,year)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [30]:
FOLDER = 'service_payments_rep_mobule'
service_payments = pd.DataFrame({
    'service_payments': 0
            },index=[0])

service_payments.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")

In [31]:
FOLDER = 'family_rate_rep_mobile'
service_payments = pd.DataFrame({
    'family_rate_rep_mobile': 2761
            },index=[0])

service_payments.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")

# Concat

In [8]:
id_count = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_ID_count/remade/{int(year)}/{int(month)}.csv', sep=';')
id_count = id_count[['Android','Ios']]
id_count.rename(columns = {'Android':'Android_id', 'Ios':'iOS_id' },inplace=True)
id_count

,Android_id,iOS_id
0,1602697,906996


In [9]:
dau = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_DAU_MAU/remade/DAU/{int(year)}/{int(month)}.csv', sep=';')
dau['DAU']= dau['Пользователи'].mean().round().astype(int)
dau = dau.loc[0,['DAU']]
dau = pd.DataFrame(data = dau.values, columns = dau.index)
dau

,DAU
0,254979


In [10]:
mau = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_DAU_MAU/remade/MAU/{int(year)}/{int(month)}.csv', sep=';')
mau['MAU'] = mau['Пользователи']
mau = mau[['MAU', 'Новые пользователи', 'Доля новых пользователей']]
mau

,MAU,Новые пользователи,Доля новых пользователей
0,760099,71075,9.350756


In [11]:
installetion_count = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_all_daownloaded/remade/{int(year)}/{int(month)}.csv', sep=';')
installetion_count = installetion_count[['Все установки', 'Сессии']]
installetion_count

,Все установки,Сессии
0,2177848,484977113


In [12]:
service_payments = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/service_payments_rep_mobule/{int(year)}/{int(month)}.csv', sep=',')
service_payments = service_payments['service_payments'].to_frame(name = 'service_payments')
service_payments

,service_payments
0,0


In [13]:
mobile_report = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/mobile_report_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
mobile_report

,Первая дата,Месяц,Год,IOS_PL,appstore_count_69,appstore_count_69_refunded,appstore_count_499,appstore_count_499_refunded,appstore_count_2390,appstore_count_2390_refunded,...,yookassa_count_35_refunded,yookassa_count_1,yookassa_count_1_refunded,yookassa_count_499,yookassa_count_499_refunded,yookassa_count_249,yookassa_count_249_refunded,yookassa_count_2390,yookassa_count_2390_refunded,refunded_amount_yookassa
0,'2024-05-01',5,2024,277017,4015,4,0,0,0,0,...,0,172,0,0,0,0,0,0,0,483


In [14]:
subscriptions_report_comerce = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/subscriptions_report_comerce_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
subscriptions_report_comerce = subscriptions_report_comerce[['paying_users','paying_users_standart','paying_users_standart_yakassa','paying_users_standart_appstore','paying_users_standart_ios_from_cart','new_subscriptions','failed_subscriptions','canceled_subscriptions','renewed_subscriptions']]
subscriptions_report_comerce

,paying_users,paying_users_standart,paying_users_standart_yakassa,paying_users_standart_appstore,paying_users_standart_ios_from_cart,new_subscriptions,failed_subscriptions,canceled_subscriptions,renewed_subscriptions
0,21481,21481,12718,3988,3622,2640,1913,316,18823


In [15]:
units_on_SK_platform = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/Units_on_SK platform_(monetization)_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
units_on_SK_platform

,Units on SK platform,Units stricted monetization,Units free monetization
0,24152,7540,16612


In [16]:
total_account_count = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/total_account_count_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
total_account_count

,total_account_count
0,1574113


In [17]:
total_activated_account = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/total_activated_account_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
total_activated_account

,total_activated_account
0,1095330


In [18]:
total_active_users = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/total_active_users_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
total_active_users

,total_active_users
0,666701


In [19]:
all_flats = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/all_flats_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
all_flats

,all_flats,freemonetization_flats,stricted_flats,subsc_restrict_is_null_flats
0,737394,690433,0,46961


In [20]:
flats_in_sputnic = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/flats_in_sputnic_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
flats_in_sputnic

,flats_in_sputnic
0,580198


In [21]:
maf = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/MAF_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
maf

,MAF,stricted_MAF,freemonetization_MAF
0,428138,0,428138


In [22]:
daf = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/DAF_rep_mobile/{int(year)}/{int(month)}.csv', sep=',').round().astype('int')
daf

,DAF,stricted_DAF,freemonetization_DAF
0,164335,0,164335


In [23]:
paying_active_flats = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/paying_active_flats_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
paying_active_flats

,paying_active_flats
0,21142


In [24]:
family_rate_rep_mobile = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/family_rate_rep_mobile/{int(year)}/{int(month)}.csv')
family_rate_rep_mobile

,family_rate_rep_mobile
0,2761


In [25]:
concated = pd.concat([dau,
                      mau, 
                      id_count,
                      installetion_count,
                      service_payments['service_payments'],
                      mobile_report,
                      subscriptions_report_comerce, 
                      units_on_SK_platform,
                      total_account_count,
                      total_activated_account,
                      total_active_users,
                      all_flats,
                      flats_in_sputnic,
                      maf,
                      daf,
                      paying_active_flats,
                      family_rate_rep_mobile
                      ]
                     , axis=1)
concated

,DAU,MAU,Новые пользователи,Доля новых пользователей,Android_id,iOS_id,Все установки,Сессии,service_payments,Первая дата,...,subsc_restrict_is_null_flats,flats_in_sputnic,MAF,stricted_MAF,freemonetization_MAF,DAF,stricted_DAF,freemonetization_DAF,paying_active_flats,family_rate_rep_mobile
0,254979,760099,71075,9.350756,1602697,906996,2177848,484977113,0,'2024-05-01',...,46961,580198,428138,0,428138,164335,0,164335,21142,2761


In [26]:
concated.to_csv(f'C:/Users/Boris/Desktop/mobile rep/Ready report/{int(year)}/{int(month)}.csv', sep=',', index=False)

In [27]:

TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow")

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'C:/Users/Boris/Desktop/mobile rep/Ready report/{int(year)}/{int(month)}.csv',
        Bucket=BUCKET_NAME,
        Key=f"mobile_report_comerce_2/year={str(year)}/{str(month)}.csv"
    )

In [28]:
get_s3_instance()
upload_dump_to_s3()
